# Hamiltonian Simulation via Taylorization

This jupyter notebook shows how to use the functions located in taylorization_lcu_circuit_generation to develop quantum circuits to simulate the Fermi-Hubbard model (or any qubit hamiltonian, really) using the Linear Combination of Unitaries (LCU) algorithm.

The methodology for constructing the circuits is mostly based on the methods discussed in the paper "Simulating Hamiltonian Dynamics with a Truncated Taylor Series" (https://arxiv.org/abs/1412.4687), with a few modifications. But despite the modifications, the end result is the same. That is, for some qubit Hamiltonian H, this algorithm implements Hamiltonian simulation for H via Taylorization. Meaning that using the LCU algorithm a quantum circuit is developed to probabilistically implement $e^{-iHt}$ using a truncated Taylor series.  


# The Model

The Fermi Hubbard model is used for describing strongly correlated systems and is thus useful for both chemistry and solid-state physics. The model describes a chain or lattice of spatial orbitals where electrons are free to hop (also called 'tunneling') from one orbital to another, and the only interactions are coulombic interactions between 2 electrons that occupy the same orbital. For a simple 1-D chain, in second quantized form the Hamiltonian that describes this system is:

$$\hat{H} = -J \sum_{i,\sigma}(\hat{c}^{\dagger}_{i,\sigma}\hat{c}_{i+1,\sigma} + \text{h.c.}) + U \sum_{i}\hat{n}_{i,\uparrow}\hat{n}_{i,\downarrow}$$

Where $i$ is the number of sites, $\sigma$ denotes the spin up or down,  and $\hat{c}^\dagger$ and $\hat{c}$ are the fermionic creation and annihilation operators respectively. $\hat{n} = \hat{c}^\dagger_{i,\sigma}\hat{c}_{i,\sigma}$ are the number operators which count if a fermion of spin sigma occupies orbital $i$. 

In this representation the first term describes the kinetic energy where $J$ is a parameter that controls how much hopping there is from one term to another, and $U$ is a parameter that determines the strength of coloumbic interactions between electrons. In this model electrons can only hop to adjacent orbitals, and for 2-D lattices cannot move diagonally.


# How to Use

To begin we first define our Hamiltonian: hubbard - by choosing the number of sites in the x and y-dimension, the tunneling J strength, the coulombic strength U and whether the system is periodic. We also choose which fermion to qubit transformation to use: Jordan_Wigner or Bravyi_Kitaev. We pass these parameters to the generate_fermi_hubbard_qubit_hamiltonian function, and the result is a qubit-Hamiltonian using the OpenFermion QubitOperator data structure.

In [1]:
from taylorization_lcu_circuit_generation import *
from openfermion import fermi_hubbard
import random
nx = 2
ny = 1
U = 2
J = 2


### defining the fermi_hubbard model
hubbard = fermi_hubbard(x_dimension=nx, y_dimension=ny, tunneling=J, coulomb=U, periodic=True)

### mapping to qubit hamiltonian

H = generate_fermi_hubbard_qubit_hamiltonian(hubbard,'Bravyi_Kitaev')

print("The Qubit Hamiltonian: ", H)

The Qubit Hamiltonian:  (1+0j) [] +
(-1+0j) [X0 Y1 Y2] +
(1+0j) [Y0 Y1 X2] +
(-0.5+0j) [Z0] +
(1+0j) [Z0 X1 Z3] +
(-0.5+0j) [Z0 Z1] +
(-1+0j) [X1 Z2] +
(0.5+0j) [Z1] +
(-0.5+0j) [Z1 Z2 Z3] +
(0.5+0j) [Z1 Z3] +
(-0.5+0j) [Z2]


We also need to define the input state, that is the wave function whose dynamics we will simulating with the defined Hamiltonian above. For simplicities sake, let's just generate some random Hartree-Fock in the occupation number basis. The representation of this state will be a bit string, and then we can generate a qiskit QuantumCircuit consisting of X gates to be our input state to our Taylorization algorithm.


In [2]:
import random
import openfermion as of
import numpy as np
from qiskit import QuantumCircuit


n_qubits = of.count_qubits(H)

def rand_key(p):
    # Variable to store the
    # string
    key1 = ""

    # Loop to find the string
    # of desired length
    for i in range(p):
        # randint function to generate
        # 0, 1 randomly and converting
        # the result into str
        temp = str(random.randint(0, 1))

        # Concatenation the random 0, 1
        # to the final result
        key1 += temp

    return (key1)


# Driver Code
n = n_qubits
str1 = rand_key(n_qubits)

random_initial_state = []
zero = np.array([1, 0])
one = np.array([0, 1])
for i in range(len(str1[::-1])):
    # print(str1[i])
    if i == 0:
        if str1[i] == '0':
            random_initial_state = np.concatenate((random_initial_state,zero), axis = 0)
        else:
            random_initial_state = np.concatenate((random_initial_state, one), axis=0)

        continue

    if str1[i] == '0':
        random_initial_state = np.kron(random_initial_state,zero)
    else:
        random_initial_state = np.kron(random_initial_state, one)



random_initial_state = np.array(random_initial_state)

print('The initial state to the Taylorization algorithm:',str1)
print('Represented as a vector in our 2^n X 2^n Hilbert Space:',random_initial_state)

test_initial_state = QuantumCircuit(n_qubits)

for i in range(len(str1)-1,-1,-1):
    # print(i)
    if str1[i] == '1':
        test_initial_state.x(i)

print('The circuit to prepare this state on a quantum computer: \n')
test_initial_state.draw()


The initial state to the Taylorization algorithm: 0001
Represented as a vector in our 2^n X 2^n Hilbert Space: [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
The circuit to prepare this state on a quantum computer: 



q_0: ─────
          
q_1: ─────
          
q_2: ─────
     ┌───┐
q_3: ┤ X ├
     └───┘

Next to do is generate the Taylor series for simulating $H$. To do this it is necessary to define the time $t$, the order of the Taylor series expansion $k$, as well as the number of time steps $r$ in the simulation. Using these values the taylor series for $e^{-\frac{iHt}{r}}$ can be approximated as $$ \widetilde{U} = \sum_{0}^{k} \frac{1}{k!} (-iHt/r)^k \approx e^{-\frac{iHt}{r}}  $$ 

Since $H$ is a linear sum of pauli words, the taylor series expansion is also a linear combination of paulis. And thus $\widetilde{U}$ is a Linear Combination of Unitaries. Thats is $\widetilde{U}$ is of the following  form $$ U_{LCU} = \sum_{0}^{J} \beta_{j}U_{j} = \widetilde{U}$$. 

First lets generate our Taylor series and then we can dive into how to actually implement this on a quantum computer.

To begin the qubit Hamiltonian $H$ and the other parameters $t$, $k$ and $r$ are  passed to the generate_taylor_series function. The result is another sequence of pauli operators that approximate $e^{-iHt/r}$.

In [17]:
from taylorization_lcu_circuit_generation import generate_taylor_series

### defining parameters
t = 1
r = 2
k = 15

taylor_series = generate_taylor_series(H, order = k, time = t,steps = r)

print('the Taylor series expansion of H: \n', taylor_series)

the Taylor series expansion of H: 
 (0.414117088525532-0.1948414514108416j) [] +
(-0.08850917747743146+0.04835278394714624j) [X0 X1 X2] +
(0.08850917747743146-0.04835278394714624j) [X0 X1 X2 Z3] +
(0.13978632360197032+0.2558771293316159j) [X0 Y1 Y2] +
(-0.037232031352892575-0.06815279517803853j) [X0 Y1 Y2 Z3] +
(-0.14796874374270827+0.08083568915166654j) [X0 Z1 X2] +
(-0.14796874374270827+0.08083568915166654j) [X0 Z1 X2 Z3] +
(0.01591181935210953+0.02912639478129619j) [X0 X2] +
(0.01591181935210953+0.02912639478129619j) [X0 X2 Z3] +
(-0.08850917747743146+0.04835278394714624j) [Y0 X1 Y2] +
(0.08850917747743146-0.04835278394714624j) [Y0 X1 Y2 Z3] +
(-0.13978632360197032-0.2558771293316159j) [Y0 Y1 X2] +
(0.037232031352892575+0.06815279517803853j) [Y0 Y1 X2 Z3] +
(-0.14796874374270827+0.08083568915166654j) [Y0 Z1 Y2] +
(-0.14796874374270827+0.08083568915166654j) [Y0 Z1 Y2 Z3] +
(0.01591181935210953+0.02912639478129619j) [Y0 Y2] +
(0.01591181935210953+0.02912639478129619j) [Y0 Y2 Z3] +
(0.

Before the circuit can be developed, we first need to do one more thing. To use the LCU algorithm, it is necesessary to write out the Taylor series expansion in the form $$U = \sum \beta_j U_j,$$ where the coefficients $\beta$ are real valued and positive. This means that the minus and complex phases in the taylor series expansion need to be absorbed into the unitaries themselves. 

To acheive this it is necessary to generate a list contain tuples of the polar decomposition of the coefficients of the unitaries to be applied. The first entry in the tuple contains $|\beta_j|$ and the second entry the angle. As well, we need a list of the pauli words, themselves, and the qubits each pauli acts on. This list will be passed to the function that generates the Select part of the LCU algorithm (more will be elaborated on that).

The code also generates a list of only the coefficient to be passed to the prepare circuit generating function (this part is actually handled in the generate_prepare_circuits function, but just so you know whats happening "under the hood").


In [18]:
from taylorization_lcu_circuit_generation import get_unitaries, get_pauliword_list, get_pauli_word_coefficient
import cmath
### getting the magnitude of the coefficients and the angles
coefficients_and_angles = []
words = get_pauliword_list(taylor_series)
for i, word in enumerate(words):
    coefficient = get_pauli_word_coefficient(word)
    coefficients_and_angles.append(tuple((coefficient,cmath.polar(coefficient)[1])))
print(coefficients_and_angles)
## we will get the list of paulis and the qubits they act on in the next step code cell


[((0.414117088525532-0.1948414514108416j), -0.43976907954277683), ((0.13978632360197032+0.2558771293316159j), 1.0707962944269154), ((-0.13978632360197032-0.2558771293316159j), -2.0707963591628777), ((-0.13978632360197032-0.2558771293316159j), -2.0707963591628777), ((0.13978632360197032+0.2558771293316159j), 1.0707962944269154), ((0.1195561431883744+0.11366217830768095j), 0.7601312353143794), ((0.1195561431883744+0.11366217830768095j), 0.7601312353143794), ((-0.13005956989181042-0.13288856747353106j), -2.3454361388897094), ((0.1195561431883744+0.11366217830768095j), 0.7601312353143794), ((0.1195561431883744+0.11366217830768095j), 0.7601312353143794), ((-0.13005956989181042-0.13288856747353106j), -2.3454361388897094), ((0.11692177803176532-0.0324829052045203j), -0.27098364049379753), ((-0.14796874374270827+0.08083568915166654j), 2.6415926742739315), ((-0.14796874374270827+0.08083568915166654j), 2.6415926742739315), ((-0.08850917747743146+0.04835278394714624j), 2.641592654310271), ((0.088

# The LCU Algorithm in Action

There are two main subroutines to call to implement the LCU algorithm for an arbitrary expansion in the type $U = \sum \beta_j U_j$, the Prepare circuit, which we will denote PREP and the Select circuit, denoted as SEL.

The prepare circuit will act on the all zero state of the ancillae register to prepare an expansion of the basis states such that $$PREP |0\rangle_{\text{anc}} = \sum \frac{\sqrt{|\beta_j|}}{\sqrt{\alpha}} |j\rangle,$$ where $\alpha = \sum |\beta_j|$

And the Select circuit will apply unitary $U_j$ controlled by state $|j\rangle$ onto input state $|\Psi\rangle$: 
$$SEL|j\rangle |\Psi \rangle = |j\rangle U_j |\Psi \rangle $$ 

The total circuit then for the LCU algorithm implementing the taylor series expansion of $e^{-iHt} |\Psi\rangle$ is: $$U_{\text{LCU}} = (PREP \otimes I)SEL(PREP^{\dagger} \otimes I)$$

Implementing this unitary $U_{LCU}$ on $|\Psi\rangle$ yields $$U_{LCU}|0\rangle_{anc} |\Psi\rangle = \frac{1}{\alpha} |0\rangle \widetilde{U} |\Psi \rangle + |1\rangle \sqrt{1-\frac{1}{s^2}}|\Phi\rangle $$

Here $|\Phi\rangle$ is some orthogonal state that we dont care about. If the ancillae state is measured to be all $|0\rangle$, then the unitary $\widetilde{U}$ that approximates $e^{-iHt}$ has been succesfully implemented probabilistically. In practice Oblivious Amplitude Amplification (OAA) is used, with careful selection of t and r, such that $\alpha = 2$, or close enough to it. However, did not have enough time to code in the OAA part, so can leave that as a project for another intern.

Another thing to note, a sum that is a linear combination of unitaries is not necessarily itself unitary. You may be wondering then, how does this method work if all allowable actions (besides measurement) on a quantum computer are unitary? That is a great question. What the LCU algorithm actually does is block encode $\frac{U}{\alpha}$ inside of a larger matrix that IS unitary.

So let's start the heart of this Taylorization algorithm by putting the prepare circuit to work!

We pass the sum of pauli words in the taylor series to the function generate prepare circuit. This function uses the qisikit 'initialize' instruction for a qiskit QuantumCircuit data type, that will prepare any arbitrary state from a provided normalized vector of amplitudes. The vector that we pass to this function is a vector whose coefficients are the magnitudes of the coefficients in the LCU expansion for the taylor series we want to implement on the quantum computer. If the length of the vector is not a power of 2, the vector is padded with zeros to make the length a power of 2. So Lets do this!

In [19]:
from taylorization_lcu_circuit_generation import generate_prepare_circuits

prepare_circuits, terms = generate_prepare_circuits(taylor_series)

prepare_circuit = prepare_circuits

## lets see what it looks like
prepare_circuit.draw()


┌─────────────┐                                                   »
q_0: ┤ Ry(0.93711) ├───────────────────────────────────────────────────»
     ├─────────────┤                                                   »
q_1: ┤ Ry(0.91598) ├───────────────────────────────────────────────────»
     └┬────────────┤                                                   »
q_2: ─┤ Ry(0.8853) ├───────────────────────────────────────────────────»
     ┌┴────────────┤                         ┌───┐┌──────────────┐┌───┐»
q_3: ┤ Ry(0.62258) ├─────────────────────────┤ X ├┤ Ry(0.015388) ├┤ X ├»
     ├─────────────┤┌───┐┌─────────────┐┌───┐└─┬─┘└──────────────┘└─┬─┘»
q_4: ┤ Ry(0.58068) ├┤ X ├┤ Ry(0.58068) ├┤ X ├──■────────────────────┼──»
     ├─────────────┤└─┬─┘└─────────────┘└─┬─┘                       │  »
q_5: ┤ Ry(0.59342) ├──■───────────────────■─────────────────────────■──»
     └─────────────┘                                                   »
«                                                                       »
«q_0: ──────────────────────────────────────────────────────────────────»
«                                                                       »
«q_1: ──────────────────────────────────────────────────────────────────»
«                                              ┌───┐┌─────────────┐┌───┐»
«q_2: ─────────────────────────────────────────┤ X ├┤ Ry(0.15041) ├┤ X ├»
«     ┌──────────────┐┌───┐┌─────────────┐┌───┐└─┬─┘└─────────────┘└─┬─┘»
«q_3: ┤ Ry(0.015388) ├┤ X ├┤ Ry(0.62258) ├┤ X ├──■───────────────────┼──»
«     └──────────────┘└─┬─┘└─────────────┘└─┬─┘                      │  »
«q_4: ──────────────────■───────────────────┼────────────────────────■──»
«                                           │                           »
«q_5: ──────────────────────────────────────■───────────────────────────»
«                                                                       »
«                                                                  »
«q_0: ─────────────────────────────────────────────────────────────»
«                                                                  »
«q_1: ─────────────────────────────────────────────────────────────»
«     ┌─────────────┐┌───┐┌─────────────┐┌───┐┌──────────────┐┌───┐»
«q_2: ┤ Ry(0.15076) ├┤ X ├┤ Ry(0.15943) ├┤ X ├┤ Ry(-0.18278) ├┤ X ├»
«     └─────────────┘└─┬─┘└─────────────┘└─┬─┘└──────────────┘└─┬─┘»
«q_3: ─────────────────■───────────────────┼────────────────────■──»
«                                          │                       »
«q_4: ─────────────────────────────────────┼───────────────────────»
«                                          │                       »
«q_5: ─────────────────────────────────────■───────────────────────»
«                                                                  »
«                                                                       »
«q_0: ──────────────────────────────────────────────────────────────────»
«                                                                  ┌───┐»
«q_1: ─────────────────────────────────────────────────────────────┤ X ├»
«     ┌──────────────┐┌───┐┌─────────────┐┌───┐┌─────────────┐┌───┐└─┬─┘»
«q_2: ┤ Ry(-0.19145) ├┤ X ├┤ Ry(-0.1918) ├┤ X ├┤ Ry(0.54309) ├┤ X ├──■──»
«     └──────────────┘└─┬─┘└─────────────┘└─┬─┘└─────────────┘└─┬─┘     »
«q_3: ──────────────────┼───────────────────■───────────────────┼───────»
«                       │                                       │       »
«q_4: ──────────────────■───────────────────────────────────────┼───────»
«                                                               │       »
«q_5: ──────────────────────────────────────────────────────────■───────»
«                                                                       »
«                                                                   »
«q_0: ──────────────────────────────────────────────────────────────»
«     ┌───────────────┐┌───┐┌───────────────┐┌───┐┌───────────┐┌───┐»
«q_1: ┤ Ry(-0.028023) ├┤ 

In the code for the prepare circuits, we can choose to transpile into any universal gate set. I had some issue with Clifford + T for some reason not working. However orquestra does have better ways to transpile into this gate set. Ask Michał for more about that. For now the prepare_circuit will generate a PREP circuit that consists of the gate set {'id', 'rx', 'ry', 'rz', 'h', 'cx'}.

The prepare circuit also returns a lists of the pauli words, in the order they were sorted in the prepare circuit function. We now call the function unitaries_and_qubits. This will generate a list of tuples containing the individual pauli operators in each pauli word as well as which qubit it needs to act on, on the input state $|\Psi\rangle$

In [20]:
unitaries_and_qubits = get_unitaries(terms)
print(unitaries_and_qubits)

[(), ((0, 'X'), (1, 'Y'), (2, 'Y')), ((0, 'Y'), (1, 'Y'), (2, 'X')), ((0, 'Z'), (1, 'X'), (3, 'Z')), ((1, 'X'), (2, 'Z')), ((0, 'Z'), (1, 'Z')), ((0, 'Z'),), ((1, 'Z'),), ((1, 'Z'), (2, 'Z'), (3, 'Z')), ((2, 'Z'),), ((1, 'Z'), (3, 'Z')), ((0, 'Z'), (2, 'Z')), ((0, 'Y'), (1, 'Z'), (2, 'Y'), (3, 'Z')), ((0, 'X'), (1, 'Z'), (2, 'X')), ((0, 'Y'), (1, 'X'), (2, 'Y')), ((0, 'X'), (1, 'X'), (2, 'X'), (3, 'Z')), ((0, 'X'), (1, 'Z'), (2, 'X'), (3, 'Z')), ((0, 'Y'), (1, 'Z'), (2, 'Y')), ((0, 'X'), (1, 'X'), (2, 'X')), ((0, 'Y'), (1, 'X'), (2, 'Y'), (3, 'Z')), ((0, 'Z'), (2, 'Z'), (3, 'Z')), ((1, 'X'), (3, 'Z')), ((0, 'Z'), (1, 'X'), (2, 'Z'), (3, 'Z')), ((0, 'Z'), (1, 'X'), (2, 'Z')), ((1, 'X'),), ((0, 'Z'), (1, 'Z'), (2, 'Z')), ((0, 'Z'), (3, 'Z')), ((0, 'Z'), (1, 'Z'), (2, 'Z'), (3, 'Z')), ((0, 'Z'), (1, 'Z'), (3, 'Z')), ((2, 'Z'), (3, 'Z')), ((1, 'Z'), (2, 'Z')), ((3, 'Z'),), ((1, 'X'), (2, 'Z'), (3, 'Z')), ((0, 'Z'), (1, 'X')), ((0, 'Y'), (1, 'Y'), (2, 'X'), (3, 'Z')), ((0, 'Y'), (2, 'Y'), (

Now that we have all this set up: the paulis in each word and the qubits they act on, the coefficients and the angles needed for their polar decomposition, and the prepare circuit that prepared the necessary amplitudes of the basis states; we can use all these ingredients to finish up the LCU circuit that will give us the means to probabilistically implement $e^{-iHt} |\Psi\rangle$. 

To do this we will use the function select_and_prep_dagger. True to its name it takes in the prepare circuit we generated earlier and adds the select circuit that contains the controlled paulis, and adds on the inverse of the prepare circuit, thus completing the LCU algorithm.



In [21]:
from taylorization_lcu_circuit_generation import select_and_prep_dagger
lcu_circuit = select_and_prep_dagger(unitaries_and_qubits, coefficients_and_angles, prepare_circuit, test_initial_state)

In [22]:
## We can print out our final circuit to see what it looks like
lcu_circuit.draw()


┌─────────────┐                                                   »
q_0: ┤ Ry(0.93711) ├───────────────────────────────────────────────────»
     ├─────────────┤                                                   »
q_1: ┤ Ry(0.91598) ├───────────────────────────────────────────────────»
     └┬────────────┤                                                   »
q_2: ─┤ Ry(0.8853) ├───────────────────────────────────────────────────»
     ┌┴────────────┤                         ┌───┐┌──────────────┐┌───┐»
q_3: ┤ Ry(0.62258) ├─────────────────────────┤ X ├┤ Ry(0.015388) ├┤ X ├»
     ├─────────────┤┌───┐┌─────────────┐┌───┐└─┬─┘└──────────────┘└─┬─┘»
q_4: ┤ Ry(0.58068) ├┤ X ├┤ Ry(0.58068) ├┤ X ├──■────────────────────┼──»
     ├─────────────┤└─┬─┘└─────────────┘└─┬─┘                       │  »
q_5: ┤ Ry(0.59342) ├──■───────────────────■─────────────────────────■──»
     └─────────────┘                                                   »
q_6: ──────────────────────────────────────────────────────────────────»
          ┌───┐     ┌───┐                                              »
q_7: ─────┤ H ├─────┤ S ├──────────────────────────────────────────────»
          ├───┤     ├───┤                                              »
q_8: ─────┤ H ├─────┤ S ├──────────────────────────────────────────────»
          ├───┤     └───┘                                              »
q_9: ─────┤ H ├────────────────────────────────────────────────────────»
          └───┘                                                        »
«                                                                       »
«q_0: ──────────────────────────────────────────────────────────────────»
«                                                                       »
«q_1: ──────────────────────────────────────────────────────────────────»
«                                              ┌───┐┌─────────────┐┌───┐»
«q_2: ─────────────────────────────────────────┤ X ├┤ Ry(0.15041) ├┤ X ├»
«     ┌──────────────┐┌───┐┌─────────────┐┌───┐└─┬─┘└─────────────┘└─┬─┘»
«q_3: ┤ Ry(0.015388) ├┤ X ├┤ Ry(0.62258) ├┤ X ├──■───────────────────┼──»
«     └──────────────┘└─┬─┘└─────────────┘└─┬─┘                      │  »
«q_4: ──────────────────■───────────────────┼────────────────────────■──»
«                                           │                           »
«q_5: ──────────────────────────────────────■───────────────────────────»
«                                                                       »
«q_6: ──────────────────────────────────────────────────────────────────»
«                                                                       »
«q_7: ──────────────────────────────────────────────────────────────────»
«                                                                       »
«q_8: ──────────────────────────────────────────────────────────────────»
«                                                                       »
«q_9: ──────────────────────────────────────────────────────────────────»
«                                                                       »
«                                                                  »
«q_0: ─────────────────────────────────────────────────────────────»
«                                                                  »
«q_1: ─────────────────────────────────────────────────────────────»
«     ┌─────────────┐┌───┐┌─────────────┐┌───┐┌──────────────┐┌───┐»
«q_2: ┤ Ry(0.15076) ├┤ X ├┤ Ry(0.15943) ├┤ X ├┤ Ry(-0.18278) ├┤ X ├»
«     └─────────────┘└─┬─┘└─────────────┘└─┬─┘└──────────────┘└─┬─┘»
«q_3: ─────────────────■───────────────────┼────────────────────■──»
«                                          │                       »
«q_4: ─────────────────────────────────────┼───────────────────────»
«                                          │                       »
«q_5: ─────────────────────────────────────■───────────────────────»
«                                                                  »
«q_6: ───────────────

And thats all you need really to generate circuits for probabilistically implementing $e^{-iHt}|\Psi\rangle$. Just a quick explanation on how the select operator works. To load in the proper sign/coefficients in the sum of the type $U = \sum \beta_j U_j$, there are two techniques which are used. if the coefficients are negative, then a controlled Rz(2$\pi$) is done first this will load in the minus sign into the unitaries being applied. If the coefficients are complex/imaginary then a controlled phase shift gate is used, achevied by using controlled phase (P($\theta$)) and controlled X gates in the sequence $P(\theta)XP(\theta)X$. Where $\theta$ if you recall is the angle derived from the polar decomposition of the complex/imaginary coefficients. 

The controlled pauli operators will either be controlled X,Y or Z. The controlled Z is implemented using the identity $HXH = Z$ and the controlled Y is implemented using $HSXS^{\dagger}H = Y$.

Now that these are all completed, lets run some tests to see if our circuit that does $e^{-iHt}|\Psi\rangle = |\Psi'\rangle$, converges to the exact solution computed clasically as we increase the order of $k$ in the taylor series for $e^{-iHt}$. The fidelity, $F$, gauges how close the two results are by computing the square magnitude of the inner product between the two $|\langle\Psi'_{exact}||\Psi'_{LCU}\rangle|^2$. The error then is just $\text{Err} = 1 -F$. If we plot log(Fidelity) and log(error) as a function of k, what we should expect to see is log(fidelity) converging to zero and exponential decay in the error. So lets make these plots now as a final test for our circuit. 

In [3]:
import scipy
from qiskit import transpile, Aer
import matplotlib.pyplot as plt
import openfermion as of
### computing exact solution classically

##Choosing simulation time, and number of time steps.
t = 1
r = 2
hamiltonian_sparse = of.get_sparse_operator(H)

exact_evolution_0 = scipy.sparse.linalg.expm_multiply(-1j * t/r * hamiltonian_sparse, random_initial_state)


### these will be the points we plot
points_fidelity = []

points_error = []

ks = []


for k in range(1, 15, 1):

    taylor_steps = generate_taylor_series(H, order = k, time = t,steps = r)
    
    ### making prepare circuit

    prepare_circuit, terms = generate_prepare_circuits(taylor_steps)

    n_ancillae = len(prepare_circuit.qubits)
    
    ## getting the list of paulis and the qubits they act on
    unitaries_and_qubits = get_unitaries(terms)

    ### Polar Decomposition allows for complex phases

    coefficients_and_angles = []
    words = get_pauliword_list(taylor_steps)
    for i, word in enumerate(words):
        coefficient = get_pauli_word_coefficient(word)
        coefficients_and_angles.append(tuple((coefficient,cmath.polar(coefficient)[1])))
        
    ### making the LCU circuit
    
    lcu_circuit = select_and_prep_dagger(unitaries_and_qubits, coefficients_and_angles, prepare_circuit, test_initial_state)


    ### Reverse ordering of bits, qiskit uses little endian, this is used for getting the 
    ### proper state vector simulating the results of the circuit
    
    lcu_circuit = lcu_circuit.reverse_bits()

    ### defining all zero initial state
    q_0 = [1,0]


    if n_ancillae == 1:
        for i in range(n_ancillae-1):
            q_0.append(0)
    else:
        for i in range(2**n_ancillae-2):
            q_0.append(0)

    q_0 = np.array(q_0)

    ## making projector |0><0|_a
    
    ancillae_projector = np.kron(np.outer(q_0,np.conjugate(q_0)),np.identity(2**n_qubits))


    exact_evolution = np.kron(q_0,exact_evolution_0)

    exact_evolution = exact_evolution/np.linalg.norm(exact_evolution)


    transpiled_circuit = transpile(lcu_circuit, basis_gates=['id', 'rx', 'ry', 'rz', 'h', 'cx'])
    backend = Aer.get_backend('statevector_simulator')
    job = backend.run(transpiled_circuit)

    result = job.result()
    output_state_3 = result.get_statevector(transpiled_circuit, decimals=3)
    
    output_state_3 = np.array(output_state_3)

    collapsed_state_0 = np.matmul(ancillae_projector,output_state_3)
    collapsed_state_1 = collapsed_state_0 ### prefactor shouldnt change cause normalize

    # breakpoint()
    collapsed_state_1 = collapsed_state_1/np.linalg.norm(collapsed_state_1)

    overlap = abs(np.inner(np.conjugate(exact_evolution),collapsed_state_1))**2

    error = 1 - overlap
    
    points_fidelity.append(np.log(overlap))
    points_error.append(np.log(error))
    ks.append(k)

###PLot should show exponential decay in the error if order is high enough/ parameters selectively chosen
plt.scatter(ks,points_fidelity)
plt.plot(ks, points_fidelity, label = 'log(Fidelity)')
plt.scatter(ks,points_error)
plt.plot(ks, points_error, label = 'log(Error)')
plt.title('H = %s, \n HF = |%s>' % (str(H),str1))
plt.xlabel('K')
plt.legend(loc="lower left")
plt.show()




1
0.04535128658715937 0.9546487134128406
2
0.011436978305584544 0.9885630216944155
3
0.0009224678011066079 0.9990775321988934
4
3.60267380310475e-05 0.999963973261969
5
1.062873317592139e-05 0.9999893712668241
6
6.90006052295189e-09 0.9999999930999395
7
6.90006052295189e-09 0.9999999930999395
8
6.90006052295189e-09 0.9999999930999395
9
6.90006052295189e-09 0.9999999930999395
10
6.90006052295189e-09 0.9999999930999395
11
6.90006052295189e-09 0.9999999930999395
12
6.90006052295189e-09 0.9999999930999395
13
6.90006052295189e-09 0.9999999930999395
14
6.90006052295189e-09 0.9999999930999395
15


KeyboardInterrupt: 